## System Setup

In [ ]:
# Find CPU info
!grep -m1 'model name' /proc/cpuinfo | awk -F": " '{print $2}'
!grep 'cpu cores' /proc/cpuinfo | awk -F": " '{a[cores]+=$2}END{print "CPU cores: " a[cores]}'

Intel(R) Xeon(R) CPU @ 2.20GHz
CPU cores: 8


In [ ]:
# Find Ram Info
!grep MemTotal /proc/meminfo | awk '{printf "%.1fGB RAM", $2 / 1024 / 1024}'

25.5GB RAM

In [ ]:
!python3 --version

Python 3.10.6


In [ ]:
!nvidia-smi

Thu Jul 20 17:52:11 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.105.17   Driver Version: 525.105.17   CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla V100-SXM2...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   37C    P0    26W / 300W |      0MiB / 16384MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
!pip install pyBKTR

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 220.3/220.3 kB 3.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 88.9/88.9 kB 8.1 MB/s eta 0:00:00


## Simulation Interpolation (Python)

In [ ]:

from pyBKTR.bktr import BKTRRegressor
from pyBKTR.kernels import KernelMatern, KernelParameter,KernelSE
from pyBKTR.tensor_ops import TSR

import numpy as np
import pandas as pd

from pyBKTR.utils import simulate_spatiotemporal_data
TSR.set_params('float32', 'cuda', 1)

res_colnames = ['Dataset_Type', 'Lengthscale', 'Iter', 'B_MAE', 'B_RMSE', 'Y_MAE', 'Y_RMSE']
nb_res_cols = len(res_colnames)

nb_aside_locs = 4
nb_aside_times = 6

df_res_arr = []

for ds_type in ['Smaller', 'Larger']:
    for len_scale in [3, 6]:
        for i in range(1, 11):
            matern_lengthscale = KernelParameter(value = len_scale)
            se_lengthscale = KernelParameter(value = len_scale)
            spatial_kernel = KernelMatern(lengthscale = matern_lengthscale, smoothness_factor = 5)
            temporal_kernel = KernelSE(lengthscale = se_lengthscale)

            nb_locs = 20 if ds_type == 'Smaller' else 100
            nb_times = 30 if ds_type == 'Smaller' else 150
            spa_cov_means = [0, 2] if ds_type == 'Smaller' else [0, 2, 4]
            tem_cov_means = [1] if ds_type == 'Smaller' else [1, 3]

            simu_data = simulate_spatiotemporal_data(
                nb_locations=nb_locs,
                nb_time_points=nb_times,
                nb_spatial_dimensions=2,
                spatial_scale=10,
                time_scale=10,
                spatial_covariates_means=spa_cov_means,
                temporal_covariates_means=tem_cov_means,
                spatial_kernel=spatial_kernel,
                temporal_kernel=temporal_kernel,
                noise_variance_scale=1
            )

            data_df = simu_data['data_df'].copy()
            spatial_positions_df = simu_data['spatial_positions_df']
            temporal_positions_df = simu_data['temporal_positions_df']

            obs_nb_locs = nb_locs - nb_aside_locs
            obs_nb_times = nb_times - nb_aside_times

            all_locs = data_df.index.get_level_values(0).unique().to_list()
            all_times = data_df.index.get_level_values(1).unique().to_list()

            locs_indx_sample = list(TSR.rand_choice(TSR.tensor(range(1, len(all_locs) + 1)), obs_nb_locs).cpu().numpy())
            obs_locs = [all_locs[int(i) - 1] for i in locs_indx_sample]
            new_locs = list(set(all_locs) - set(obs_locs))

            times_indx_sample = list(TSR.rand_choice(TSR.tensor(range(1, len(all_times) + 1)), obs_nb_times).cpu().numpy())
            obs_times = [all_times[int(i) - 1] for i in times_indx_sample]
            new_times = list(set(all_times) - set(obs_times))

            obs_data_df = data_df.drop(index=new_locs, level='location')
            obs_data_df = obs_data_df.drop(index=new_times, level='time')
            obs_spatial_pos_df = spatial_positions_df.drop(index=new_locs,)
            obs_temporal_pos_df = temporal_positions_df.drop(index=new_times,)

            new_data_df = data_df[
                (data_df.index.get_level_values(0).isin(new_locs)) |
                (data_df.index.get_level_values(1).isin(new_times))
            ]
            new_beta_data_df = simu_data['beta_df'].copy()
            new_beta_data_df = new_beta_data_df[
                (new_beta_data_df.index.get_level_values(0).isin(new_locs)) |
                (new_beta_data_df.index.get_level_values(1).isin(new_times))
            ]
            new_spatial_pos_df = spatial_positions_df[spatial_positions_df.index.isin(new_locs)]
            new_temporal_pos_df = temporal_positions_df[temporal_positions_df.index.isin(new_times)]


            bktr_regressor = BKTRRegressor(
                data_df = obs_data_df,
                rank_decomp = 10,
                burn_in_iter = 500,
                sampling_iter = 500,
                spatial_kernel = KernelMatern(smoothness_factor = 5),
                spatial_positions_df = obs_spatial_pos_df,
                temporal_kernel = KernelSE(),
                temporal_positions_df = obs_temporal_pos_df,
                has_geo_coords = False
            )
            bktr_regressor.mcmc_sampling()

            preds_y_df, preds_beta_df = bktr_regressor.predict(new_data_df, new_spatial_pos_df, new_temporal_pos_df)

            preds_y_df.sort_index(inplace=True)
            new_data_df.sort_index(inplace=True)
            preds_beta_df.sort_index(inplace=True)
            new_beta_data_df.sort_index(inplace=True)
            preds_y_err = new_data_df['y'].to_numpy() - preds_y_df['y'].to_numpy()
            preds_beta_err = new_beta_data_df.to_numpy() - preds_beta_df.to_numpy()
            df_res_arr.append([
                ds_type,
                len_scale,
                i,
                np.mean(np.abs(preds_beta_err)),
                np.sqrt(np.mean(np.square(preds_beta_err))),
                np.mean(np.abs(preds_y_err)),
                np.sqrt(np.mean(np.square(preds_y_err))),
            ])
            df_res = pd.DataFrame(df_res_arr, columns=res_colnames)
            print(df_res)



/usr/local/lib/python3.10/dist-packages/torch/distributions/wishart.py:253: UserWarning: Singular sample detected.
  warnings.warn("Singular sample detected.")


* Iter 1     | Elapsed     0.40s | MAE  0.7274 | RMSE  0.9429 *
* Iter 2     | Elapsed     0.04s | MAE  0.7697 | RMSE  0.9582 *
* Iter 3     | Elapsed     0.03s | MAE  0.7709 | RMSE  0.9493 *
* Iter 4     | Elapsed     0.02s | MAE  0.7790 | RMSE  0.9841 *
* Iter 5     | Elapsed     0.03s | MAE  0.7863 | RMSE  0.9883 *
* Iter 6     | Elapsed     0.03s | MAE  0.8384 | RMSE  1.0489 *
* Iter 7     | Elapsed     0.02s | MAE  0.8317 | RMSE  1.0445 *
* Iter 8     | Elapsed     0.04s | MAE  0.7776 | RMSE  0.9827 *
* Iter 9     | Elapsed     0.04s | MAE  0.7873 | RMSE  1.0046 *
* Iter 10    | Elapsed     0.03s | MAE  0.7926 | RMSE  1.0019 *
* Iter 11    | Elapsed     0.02s | MAE  0.8028 | RMSE  1.0282 *
* Iter 12    | Elapsed     0.03s | MAE  0.7687 | RMSE  0.9995 *
* Iter 13    | Elapsed     0.03s | MAE  0.8273 | RMSE  1.0452 *
* Iter 14    | Elapsed     0.04s | MAE  0.7863 | RMSE  0.9884 *
* Iter 15    | Elapsed     0.06s | MAE  0.7688 | RMSE  0.9865 *
* Iter 16    | Elapsed     0.03s | MAE  

/usr/local/lib/python3.10/dist-packages/pyBKTR/bktr.py:856: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.sort_index(inplace=True)
<ipython-input-6-c5019a6fb8f9>:97: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_data_df.sort_index(inplace=True)
/usr/local/lib/python3.10/dist-packages/torch/distributions/wishart.py:253: UserWarning: Singular sample detected.
  warnings.warn("Singular sample detected.")


  Dataset_Type  Lengthscale  Iter    B_MAE    B_RMSE     Y_MAE    Y_RMSE
0      Smaller            3     1  0.79141  1.001138  1.378541  1.782494
* Iter 1     | Elapsed     0.02s | MAE  0.9705 | RMSE  1.2592 *
* Iter 2     | Elapsed     0.02s | MAE  0.8968 | RMSE  1.1365 *
* Iter 3     | Elapsed     0.02s | MAE  0.7832 | RMSE  0.9919 *
* Iter 4     | Elapsed     0.02s | MAE  0.7760 | RMSE  0.9853 *
* Iter 5     | Elapsed     0.02s | MAE  0.7730 | RMSE  0.9663 *
* Iter 6     | Elapsed     0.02s | MAE  0.7800 | RMSE  0.9791 *
* Iter 7     | Elapsed     0.03s | MAE  0.7932 | RMSE  1.0133 *
* Iter 8     | Elapsed     0.02s | MAE  0.7900 | RMSE  0.9951 *
* Iter 9     | Elapsed     0.04s | MAE  0.7656 | RMSE  0.9646 *
* Iter 10    | Elapsed     0.03s | MAE  0.8288 | RMSE  1.0333 *
* Iter 11    | Elapsed     0.02s | MAE  0.7776 | RMSE  0.9950 *
* Iter 12    | Elapsed     0.03s | MAE  0.8188 | RMSE  1.0049 *
* Iter 13    | Elapsed     0.02s | MAE  0.7635 | RMSE  0.9641 *
* Iter 14    | Elapsed

/usr/local/lib/python3.10/dist-packages/pyBKTR/bktr.py:856: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.sort_index(inplace=True)
<ipython-input-6-c5019a6fb8f9>:97: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_data_df.sort_index(inplace=True)
/usr/local/lib/python3.10/dist-packages/torch/distributions/wishart.py:253: UserWarning: Singular sample detected.
  warnings.warn("Singular sample detected.")


...

/usr/local/lib/python3.10/dist-packages/pyBKTR/bktr.py:856: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.sort_index(inplace=True)
<ipython-input-6-c5019a6fb8f9>:97: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_data_df.sort_index(inplace=True)
/usr/local/lib/python3.10/dist-packages/torch/distributions/wishart.py:253: UserWarning: Singular sample detected.
  warnings.warn("Singular sample detected.")


   Dataset_Type  Lengthscale  Iter     B_MAE    B_RMSE     Y_MAE    Y_RMSE
0       Smaller            3     1  0.791410  1.001138  1.378541  1.782494
1       Smaller            3     2  0.453882  0.596162  1.505989  2.062713
2       Smaller            3     3  0.680061  0.890005  1.361288  1.778193
3       Smaller            3     4  0.525499  0.687352  1.131098  1.476524
4       Smaller            3     5  0.747671  0.993836  1.498948  1.996148
5       Smaller            3     6  0.825085  1.064435  1.372535  1.725401
6       Smaller            3     7  0.974488  1.436641  1.529864  2.126434
7       Smaller            3     8  0.719792  1.032030  1.932401  2.474643
8       Smaller            3     9  1.084566  1.425300  2.758249  4.654967
9       Smaller            3    10  0.819045  1.151139  2.299390  3.433995
10      Smaller            6     1  0.549024  0.685846  1.157535  1.520684
11      Smaller            6     2  0.395283  0.520298  0.918409  1.122545
12      Smaller          

/usr/local/lib/python3.10/dist-packages/pyBKTR/bktr.py:856: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.sort_index(inplace=True)


   Dataset_Type  Lengthscale  Iter     B_MAE    B_RMSE     Y_MAE    Y_RMSE
0       Smaller            3     1  0.791410  1.001138  1.378541  1.782494
1       Smaller            3     2  0.453882  0.596162  1.505989  2.062713
2       Smaller            3     3  0.680061  0.890005  1.361288  1.778193
3       Smaller            3     4  0.525499  0.687352  1.131098  1.476524
4       Smaller            3     5  0.747671  0.993836  1.498948  1.996148
5       Smaller            3     6  0.825085  1.064435  1.372535  1.725401
6       Smaller            3     7  0.974488  1.436641  1.529864  2.126434
7       Smaller            3     8  0.719792  1.032030  1.932401  2.474643
8       Smaller            3     9  1.084566  1.425300  2.758249  4.654967
9       Smaller            3    10  0.819045  1.151139  2.299390  3.433995
10      Smaller            6     1  0.549024  0.685846  1.157535  1.520684
11      Smaller            6     2  0.395283  0.520298  0.918409  1.122545
12      Smaller          

<ipython-input-6-c5019a6fb8f9>:97: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_data_df.sort_index(inplace=True)


In [ ]:
from google.colab import runtime
runtime.unassign()